In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt install libomp-dev
!pip install faiss
!pip install faiss-gpu
!pip install elasticsearch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (873 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 121654 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a14

In [ ]:
!pip install annoy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552448 sha256=ac0956661de20e7474c221312ca513d97d94e79e57d83ed8a46cb863261eff58
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

from sklearn.neighbors import NearestNeighbors

from PIL import Image
import math
import numpy as np
from sklearn.utils import shuffle
import os
import json
from sklearn.model_selection import train_test_split
import cv2
from google.colab.patches import cv2_imshow
from scipy import spatial

import re

##Load Data

In [ ]:
data_path = "/content/drive/MyDrive/data/paris_index.json"
with open(data_path) as fp:
    data = json.load(fp)

In [ ]:
paths = data['paths']
names = data['names']
vectors = np.array(data['vectors'])

In [ ]:
vectors = vectors.astype(np.float32)

In [ ]:
vectors.shape

(6392, 512)

##Faiss

In [ ]:
import faiss
flower_index = faiss.IndexFlatL2(vectors.shape[1])
flower_index.add(vectors)

##Load query

In [ ]:
def extract_file_names(path):
    listdir = os.listdir(path)
    list_name = []
    for file in listdir:
        type_file = file.split('_')[-1][:-4]
        if type_file == 'good':
            list_name.append(file[:-9])
    return list_name

In [ ]:
query_file_path = "/content/drive/MyDrive/data/query_paris"
query_file = extract_file_names(query_file_path)

In [ ]:
len(query_file)

55

In [ ]:
query_file[:5]

['paris_defense_1',
 'paris_defense_4',
 'paris_defense_2',
 'paris_defense_3',
 'paris_defense_5']

In [ ]:
path = "/content/drive/MyDrive/data/paris_index.json"
with open(path) as fp:
    query = json.load(fp)

In [ ]:
query['names'][:5]

['paris_pompidou_000627',
 'paris_pompidou_000629',
 'paris_pompidou_000631',
 'paris_pompidou_000633',
 'paris_pompidou_000636']

In [ ]:
def remove_trailing_zeros(input_string):
    parts = input_string.rsplit('_', 1)

    if len(parts) == 2:
        prefix, number_part = parts
        modified_number = re.sub(r'^0+', '', number_part)
        modified_string = f'{prefix}_{modified_number}'
        return modified_string

    return input_string

In [ ]:
vectors_query = {}
for i in range(len(query['names'])):
    original_string = query['names'][i]
    modified_string = remove_trailing_zeros(original_string)
    print(modified_string)
    vectors_query[modified_string] = query['vectors'][i]


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
paris_pantheon_702
paris_pantheon_706
paris_pantheon_709
paris_pantheon_711
paris_pantheon_713
paris_pantheon_714
paris_pantheon_715
paris_pantheon_721
paris_pantheon_722
paris_pantheon_723
paris_pantheon_726
paris_pantheon_728
paris_pantheon_730
paris_pantheon_732
paris_pantheon_740
paris_pantheon_741
paris_pantheon_744
paris_pantheon_748
paris_pantheon_749
paris_pantheon_753
paris_pantheon_754
paris_pantheon_755
paris_pantheon_756
paris_pantheon_759
paris_pantheon_762
paris_pantheon_765
paris_pantheon_770
paris_pantheon_773
paris_pantheon_776
paris_pantheon_780
paris_pantheon_781
paris_pantheon_783
paris_pantheon_789
paris_pantheon_792
paris_pantheon_796
paris_pantheon_799
paris_pantheon_802
paris_pantheon_810
paris_pantheon_811
paris_pantheon_814
paris_pantheon_818
paris_pantheon_819
paris_pantheon_820
paris_pantheon_821
paris_pantheon_823
paris_pantheon_824
paris_pantheon_827
paris_pantheon_829
paris_pantheon_831
paris_panthe

##List rank for each query

In [ ]:
import faiss
import numpy as np
from scipy import spatial


# vectors = np.array(data['vectors'])
def re_ranking(initial_results, distance):

    combined_results = list(zip(initial_results, distance, range(len(initial_results))))

    # Sort the combined results by the refined distance metric (you can customize this metric)
    combined_results.sort(key=lambda x: (x[1], -x[2]))

    # Extract the re-ranked image IDs
    re_ranked_results = [result[0] for result in combined_results]

    return re_ranked_results

def search(index, names, query_vector, k, data_vectors = vectors):
    # Convert the query vector to float32 if needed
    distances = [spatial.distance.cosine(i, query_vector) for i in data_vectors]
    ids = np.argsort(distances)[:k]
    results = [data['paths'][id] for id in ids]
    results = re_ranking(results,distances)

    return results, distances


In [ ]:
def search1(index, names, query_vector, k):
    distances, indices = index.search(np.expand_dims(query_vector, axis=0), k)

    # Extract the names of the nearest neighbors
    nearest_paths = [names[i] for i in indices[0]]

    return nearest_paths, distances

In [ ]:
from annoy import AnnoyIndex
import numpy as np

def build_annoy_index(vectors = data['vectors'], metric='angular'):
    # Assuming vectors is a list of vectors
    dimension = len(vectors[0])
    index = AnnoyIndex(dimension, metric=metric)

    for i, vector in enumerate(vectors):
        index.add_item(i, vector)

    index.build(n_trees=10)  # You can adjust the number of trees for performance

    return index

def search_annoy(index, query_vector, k):
    # Perform a search
    indices = index.get_nns_by_vector(query_vector, k)

    return indices

        # index = build_annoy_index()
        # result_indices = search_annoy(index, query_vector, k=5000)
        # nearest_paths = [data['names'][i] for i in result_indices]

In [ ]:
image_path = "/content/drive/MyDrive/data/data_paris/"
for name in query_file:
    if name in vectors_query:
        query_vector = np.array(vectors_query[name])
        query_vector = query_vector.astype(np.float32)
        nearest_paths, nearest_distances = search(flower_index, names, query_vector, 5000)
        list_rank_path = "/content/drive/MyDrive/data/paris_list_rank/" + name + ".json"
        with open(list_rank_path, "w") as fp:
            json.dump(nearest_paths, fp, indent=4)

In [ ]:
# from sklearn.neighbors import NearestNeighbors
# import json
# import numpy as np

# # Assuming vectors_query is a dictionary containing vector representations for each image name

# # Extract vectors and corresponding names


# # Create a NearestNeighbors model with Euclidean distance (default)
# neighbors_model = NearestNeighbors(n_neighbors=5000, algorithm='auto', metric='euclidean')

# # Fit the model with your vectors
# neighbors_model.fit(vectors)

# # Perform search for each query
# for name in query_file:
#     if name in vectors_query:
#         query_vector = np.array(vectors_query[name]).reshape(1, -1)  # Reshape to match the input format
#         distances, indices = neighbors_model.kneighbors(query_vector)

#         # Get the nearest paths
#         nearest_paths = [names[index] for index in indices.flatten()]

#         # Save the results to a JSON file
#         list_rank_path = "/content/drive/MyDrive/data/paris_list_rank/" + name + ".json"
#         with open(list_rank_path, "w") as fp:
#             json.dump(nearest_paths, fp, indent=4)
#     else:
#         print(f"Vector for name {name} not found. Skipping...")





##Evaluation

###MAP

In [ ]:
k = 2000

list_rank_path = "/content/drive/MyDrive/data/paris_list_rank/"
query_file_path = "/content/drive/MyDrive/data/query_paris/"
list_ap = []
for name in query_file:
    try:
        with open(list_rank_path + name + ".json") as fp:
            list_rank = json.load(fp)
    except FileNotFoundError:
        print(f"File not found: {list_rank_path + name + '.json'}. Ignoring...")
        continue
    with open(query_file_path + name + "_good.txt", 'r') as fp:
        good_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_ok.txt", 'r') as fp:
        ok_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_junk.txt", 'r') as fp:
        junk_set = fp.read().split('\n')[:-1]
    right_set = good_set + ok_set

    pos = len(right_set)
    ap = 0
    relevant = 0
    old_recall = 0
    old_pre = 1
    j = 0
    for i in range(k):
        if any(s in list_rank[i] for s in right_set):
            relevant += 1
        elif any(s in list_rank[i] for s in junk_set):
            continue
        pre = relevant / (j + 1)
        recall = relevant/pos
        ap += (recall - old_recall)*((pre + old_pre)/2)
        j += 1
        old_recall = recall
        old_pre = pre
    list_ap.append(np.mean(ap))
map = np.mean(list_ap)
print(map)

File not found: /content/drive/MyDrive/data/paris_list_rank/paris_defense_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_defense_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_eiffel_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_eiffel_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_invalides_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_louvre_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_invalides_5.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_moulinrouge_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_louvre_5.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_louvre_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_ra

###MAP@K

In [ ]:
k = 5
list_rank_path = "/content/drive/MyDrive/data/paris_list_rank/"
query_file_path = "/content/drive/MyDrive/data/query_paris/"
ap = []
for name in query_file:
    try:
        with open(list_rank_path + name + ".json") as fp:
            list_rank = json.load(fp)
    except FileNotFoundError:
        print(f"File not found: {list_rank_path + name + '.json'}. Ignoring...")
        continue
    with open(query_file_path + name + "_good.txt", 'r') as fp:
        good_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_ok.txt", 'r') as fp:
        ok_set = fp.read().split('\n')[:-1]
    with open(query_file_path + name + "_junk.txt", 'r') as fp:
        junk_set = fp.read().split('\n')[:-1]
    right_set = good_set + ok_set

    precision = []
    relevant = 0
    j = 0
    for i in range(k):
        if any(s in list_rank[i] for s in right_set):
            relevant += 1
        elif any(s in list_rank[i] for s in junk_set):
            continue
        pre = relevant / (j + 1)
        precision.append(pre)
        j += 1
    ap.append(np.mean(precision))
map = np.mean(ap)
print('top ' + str(k) + ': ', map)

File not found: /content/drive/MyDrive/data/paris_list_rank/paris_defense_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_defense_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_eiffel_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_eiffel_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_invalides_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_louvre_3.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_invalides_5.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_moulinrouge_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_louvre_5.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_rank/paris_louvre_1.json. Ignoring...
File not found: /content/drive/MyDrive/data/paris_list_ra

In [ ]:
ap

[0.00742905565938798,
 0.00359738888756347,
 0.0,
 0.032396923292230186,
 0.008870652913455918,
 0.009882004232837428,
 0.0,
 0.0,
 0.011994869083211077,
 0.03602696390944565,
 0.0,
 0.0,
 0.0,
 0.0,
 0.007708311444261844,
 0.0,
 0.0,
 0.01520232300495071,
 0.0,
 0.01732985384082585,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.15172666644617927,
 0.15172666644617927,
 0.12145577683255268,
 0.2959723746828656,
 0.02412541105758519,
 0.11647705760258815,
 0.01608963445126837,
 0.011972605046470235]

##Visualize Result